In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk
import textblob
import gensim
from PIL import Image

from gensim import corpora
from gensim.models import LdaModel
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import Word
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, ImageColorGenerator
from gensim import similarities
from gensim import models
from nltk.stem.porter import *

In [2]:
df = pd.read_excel("amazon_review_processed_full.xlsx")
df.head(5)

,Review Model,Retailer,Review date,Review name,Review rating,Review title,Review Content,Verified Purchase or not,People_find_helpful,vine or not,...,title_word_count,content_char_count,title_char_count,content_avg_word,title_avg_word,content_stopwords,title_stopwords,Brand,Full review,ID
0,Canon Pixma TS6420a,Amazon,2023-08-10,Ernest Birkholz,5,works great,easy install works great,True,0.0,False,...,2,38,11,3.875000,5.000000,3,0,Canon,works great easy install works great,1
1,HP OfficeJet Pro 9015e,Amazon,2022-06-04,mattey,3,spunky mid size,slower speed im used old model 8600 new 9015e ...,True,0.0,False,...,4,460,23,4.835443,5.000000,19,0,HP,spunky mid size slower speed im used old model...,2
2,Canon PIXMA MG3620,Amazon,2023-03-15,Maria D,4,NaN,i,True,0.0,False,...,2,7,12,3.000000,5.500000,0,0,Canon,i,3
3,Epson - ET-3830,Amazon,2022-11-17,Ryan H,5,shag knows hes talking,yup slam upgrade paid little front equivalent ...,True,12.0,False,...,6,469,34,4.595238,4.833333,31,2,Epson,shag knows hes talking yup slam upgrade paid l...,4
4,HP ENVY 6055e,Amazon,2022-04-09,Sam,3,user friendly,would think something simple reconnecting wifi...,True,0.0,False,...,3,861,17,4.387500,5.000000,71,0,HP,user friendly would think something simple rec...,5


In [5]:
# Tokenisation (Full Review)
df['Tokenized Full review'] = df['Full review'].apply(lambda x: word_tokenize(str(x)) if isinstance(x, str) else [])
df[['Full review', 'Tokenized Full review']].head(5)

,Full review,Tokenized Full review
0,works great easy install works great,"[works, great, easy, install, works, great]"
1,spunky mid size slower speed im used old model...,"[spunky, mid, size, slower, speed, im, used, o..."
2,i,[i]
3,shag knows hes talking yup slam upgrade paid l...,"[shag, knows, hes, talking, yup, slam, upgrade..."
4,user friendly would think something simple rec...,"[user, friendly, would, think, something, simp..."


In [13]:
# Create a dictionary from the tokenized content
dictionary = corpora.Dictionary(df['Tokenized Full review'])

# Create a corpus (bag of words) from the tokenized content
corpus = [dictionary.doc2bow(text) for text in df['Tokenized Full review']]
# To find the similarity scores, create a reverse index
Index = similarities.SparseMatrixSimilarity(corpus, len(dictionary))

# Create a TFIDF reveset index
TFIDF = models.TfidfModel(corpus)
corpus_TFIDF = [TFIDF[vec] for vec in corpus]
Index_TFIDF = similarities.SparseMatrixSimilarity(corpus_TFIDF, len(dictionary))

In [30]:
# A Query string
query = input("What would you like to search for? ")
qList = query.split()
qLower = [w.lower() for w in qList]

# Stem it 
stemmer = PorterStemmer()
qStemmed = [stemmer.stem(w) for w in qLower]

# Create a query vector using the same dicitonary as the corpus
qVector = dictionary.doc2bow(qStemmed)

# Get its TFIDF from the same model as the corpus
qVectorTFIDF = TFIDF[qVector]
# print(qVector, qVectorTFIDF)

# Get the similaries from the two indexes (raw and TFIDF)
simRaw = Index[qVector]
simTFIDF = Index_TFIDF[qVectorTFIDF]

# Set display options to show the entire review content
pd.set_option('display.max_colwidth', None)

# Create a DataFrame to store the results
results_raw = pd.DataFrame({
    'Document': range(len(df)),
    'Raw Similarity Score': simRaw
})

results_tfidf = pd.DataFrame({
    'Document': range(len(df)),
    'TFIDF Similarity Score': simTFIDF
})

# Add the Similarity Score as a new column to the original DataFrame
df['Raw Similarity Score'] = results_raw['Raw Similarity Score']
df['TFIDF Similarity Score'] = results_tfidf['TFIDF Similarity Score']

# Sort the DataFrame by Raw Similarity Score in descending order
df_sorted_raw = df.sort_values(by='Raw Similarity Score', ascending=False)
df_sorted_tfidf = df.sort_values(by='TFIDF Similarity Score', ascending=False)

# Display the sorted DataFrame
#df_sorted_raw.head(10)
df_sorted_tfidf.head(10)


What would you like to search for? low quality


,Review Model,Retailer,Review date,Review name,Review rating,Review title,Review Content,Verified Purchase or not,People_find_helpful,vine or not,...,content_avg_word,title_avg_word,content_stopwords,title_stopwords,Brand,Full review,ID,Tokenized Full review,Raw Similarity Score,TFIDF Similarity Score
686,Canon PIXMA TS3520,Amazon,2022-11-13,KKHARVEY,5,good quality low price,pleased good quality low price,True,0.0,False,...,4.700000,4.75,3,0,Canon,good quality low price pleased good quality low price,687,"[good, quality, low, price, pleased, good, quality, low, price]",0.485071,0.660851
3187,HP DeskJet 2755e,Amazon,2023-08-17,irish57,5,low price,setup took time,True,0.0,False,...,4.333333,4.00,0,0,HP,low price setup took time,3188,"[low, price, setup, took, time]",0.447214,0.605324
1627,HP Smart Tank 5101,Amazon,2023-09-10,William walker,5,works,worth see gets low,True,0.0,False,...,3.117647,4.50,9,0,HP,works worth see gets low,1628,"[works, worth, see, gets, low]",0.447214,0.494961
1593,HP ENVY 6055e,Amazon,2021-12-03,Rebeca Montealegre,3,low quality,disconnects time spend lot time trying came toner reading low really frustrating trying,True,0.0,False,...,4.270270,5.00,16,0,HP,low quality disconnects time spend lot time trying came toner reading low really frustrating trying,1594,"[low, quality, disconnects, time, spend, lot, time, trying, came, toner, reading, low, really, frustrating, trying]",0.436436,0.474494
5238,Canon PIXMA TR4720,Amazon,2023-08-16,Sindy,2,low quality,easy install job sucked decided up slides first time quality low,True,0.0,False,...,3.862069,6.00,13,0,Canon,low quality easy install job sucked decided up slides first time quality low,5239,"[low, quality, easy, install, job, sucked, decided, up, slides, first, time, quality, low]",0.485071,0.467867
7029,Canon PIXMA TR8620a,Amazon,2023-03-30,LJ,5,great takes low priced cartridges,office used liked much bought home office printers used lot love low cost cartridges available form amazon,True,2.0,False,...,3.900000,5.50,14,1,Canon,great takes low priced cartridges office used liked much bought home office printers used lot love low cost cartridges available form amazon,7030,"[great, takes, low, priced, cartridges, office, used, liked, much, bought, home, office, printers, used, lot, love, low, cost, cartridges, available, form, amazon]",0.365148,0.416413
2041,Canon PIXMA TR4720,Amazon,2023-01-15,Kontji Anthony,1,buy buy every 20 pages,third last purchase gone pack dont lot 20 pages got low message bought new immediately said low super busy didnt time report third time ordered immediately said low defective product total loss,True,2.0,False,...,3.987013,3.75,24,1,Canon,buy buy every 20 pages third last purchase gone pack dont lot 20 pages got low message bought new immediately said low super busy didnt time report third time ordered immediately said low defective product total loss,2042,"[buy, buy, every, 20, pages, third, last, purchase, gone, pack, dont, lot, 20, pages, got, low, message, bought, new, immediately, said, low, super, busy, didnt, time, report, third, time, ordered, immediately, said, low, defective, product, total, loss]",0.397360,0.394688
5316,Epson - ET-3830,Amazon,2023-09-02,Javier,5,great,prints high quality cost per copy extremely low,True,0.0,False,...,4.153846,6.00,4,0,Epson,great prints high quality cost per copy extremely low,5317,"[great, prints, high, quality, cost, per, copy, extremely, low]",0.333333,0.371003
168,HP Smart Tank 6001,Amazon,2023-05-11,Jennifer Buck,5,great,works great love fact dont buy cartridges refill gets low,True,0.0,False,...,3.695652,6.00,12,0,HP,great works great love fact dont buy cartridges refill gets low,169,"[great, works, great, love, fact, dont, buy, cartridges, refill, gets, low]",0.277350,0.360030
4299,HP ENVY Inspire 7955e,Amazon,2022-01-20,Amazon Customer,5,nice,nice quality low price purchased based family recommendations quality great easy like says getting low waiting would nice come full size